In [ ]:
import warnings
warnings.simplefilter('always', DeprecationWarning)

from osgeo import gdal
from osgeo import ogr
import numpy as np
# from osgeo import gdal_array
# from osgeo import gdalnumeric

import os
import contextlib

gdal.UseExceptions()

def silent_remove(path):
    with contextlib.suppress(FileNotFoundError):
        os.remove(path)

## Lecture de l'ENC

In [ ]:
enc_ds = ogr.Open('data/ENC_ROOT/US4MA13M/US4MA13M.000')
assert enc_ds.GetDriver().GetDescription() == 'S57' # S57
m_covr_lyr = enc_ds.GetLayerByName('M_COVR')
soundg_lyr = enc_ds.GetLayerByName('SOUNDG')

covr_geo = [f.GetGeometryRef().Clone() for f in m_covr_lyr if f.GetField('CATCOV') == 1][0]
covr_geo

In [ ]:
json_drv = ogr.GetDriverByName('GeoJSON')
silent_remove('data/covr.geojson')
covr_ds = json_drv.CreateDataSource('data/covr.geojson')
covr_lyr = covr_ds.CreateLayer('covr')
featureDefn = covr_lyr.GetLayerDefn()
featureDefn.AddFieldDefn(ogr.FieldDefn('NAME', ogr.OFTString))
covr_f = ogr.Feature(featureDefn)

In [ ]:
covr_f.SetFieldString(1,'Hello')

In [ ]:
#covr_f.SetGeometry(covr_geo)
covr_f.SetGeometry(ogr.CreateGeometryFromWkt('POINT(2 3)'))

In [ ]:
covr_lyr.CreateFeature(covr_f)

In [ ]:
covr_ds.Destroy()

## Lecture de la Bathy

In [ ]:
bathy_ds = gdal.Open('data/navd_bath_30m')
assert bathy_ds.GetDriver().GetDescription() == 'AIG' # ArcInfo Binary Grid
assert bathy_ds.RasterCount == 1
bathy_band = bathy_ds.GetRasterBand(1)
bathy_np = bathy_band.ReadAsArray()
# bathy_np[bathy_np == bathy_band.GetNoDataValue()] = np.NaN

In [ ]:
bathy_np.shape

In [ ]:
for f in m_covr_lyr:
    if f.GetField('CATCOV') != 22:
        g = f.GetGeometryRef()
        print(g.GetEnvelope())
        #print(g.ExportToWkt())

In [ ]:
g.Clone()

In [ ]:
geotransform = bathy_ds.GetGeoTransform()
print("Origin = ({}, {})".format(geotransform[0], geotransform[3]))
print("Pixel Size = ({}, {})".format(geotransform[1], geotransform[5]))

## Essais de GDAL Warp

In [ ]:
gdal.Warp("test3.gpkg", 'data/navd_bath_30m', options = gdal.WarpOptions(format='GPKG'))

In [ ]:
out = gdal.Warp("test6.tif", 'data/navd_bath_30m', options = gdal.WarpOptions(format='GTiff', cutlineDSName='data/ENC_ROOT/US4MA13M/US4MA13M.000', cutlineLayer='M_COVR',))

In [ ]:
out.FlushCache()

In [ ]:
out = gdal.Warp("test5.tif", 'data/navd_bath_30m')

In [ ]:
out.FlushCache()